In [51]:
import os
import numpy as np
import pretty_midi
from mido import MidiFile, MidiTrack, Message
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, LeakyReLU, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [2]:
def read_midi(file_path):
    try:
        midi = pretty_midi.PrettyMIDI(file_path)
        notes = []
        for instrument in midi.instruments:
            for note in instrument.notes:
                notes.append((note.pitch, note.start))
        return notes
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return []


In [3]:
def load_midi_files(folder_path):
    all_notes = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.mid'):
                file_path = os.path.join(root, file)
                notes = read_midi(file_path)
                if notes:  # Only add if notes were successfully read
                    all_notes.append(notes)
    return all_notes

In [4]:
midi_folder_path = "C:/Users/Student_B21/Downloads/Model"
all_midi_notes = load_midi_files(midi_folder_path)

C:\Users\Student_B21\Documents\Custom Office Templates\Python\Lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Error reading C:/Users/Student_B21/Downloads/Model\10cc\Dreadlock_Holiday.4.mid: data byte must be in range 0..127
Error reading C:/Users/Student_B21/Downloads/Model\3T\Why.mid: data byte must be in range 0..127
Error reading C:/Users/Student_B21/Downloads/Model\4_Non_Blondes\Whats_Up.5.mid: data byte must be in range 0..127
Error reading C:/Users/Student_B21/Downloads/Model\a-ha\Take_On_Me.1.mid: data byte must be in range 0..127
Error reading C:/Users/Student_B21/Downloads/Model\Aaron_Neville\Tell_It_Like_It_Is.mid: data byte must be in range 0..127
Error reading C:/Users/Student_B21/Downloads/Model\ABBA\Ive_Been_Waiting_For_You.mid: data byte must be in range 0..127
Error reading C:/Users/Student_B21/Downloads/Model\ABBA\One_Of_Us.mid: data byte must be in range 0..127
Error reading C:/Users/Student_B21/Downloads/Model\ABBA\Take_a_Chance_on_Me.3.mid: data byte must be in range 0..127
Error reading C:/Users/Student_B21/Downloads/Model\ABBA\Thank_You_for_the_Music.2.mid: data byte mus

In [39]:
def normalize_notes(all_notes):
    all_notes_flat = [note for sublist in all_notes for note in sublist]
    min_note = min(all_notes_flat, key=lambda x: x[0])[0]
    max_note = max(all_notes_flat, key=lambda x: x[0])[0]
    normalized_notes = [(note[0] - min_note) / (max_note - min_note) for note in all_notes_flat]
    return np.array(normalized_notes).reshape(-1, 1), min_note, max_note  # Return three values

normalized_notes, min_note, max_note = normalize_notes(all_midi_notes)

In [40]:
def build_generator(latent_dim):
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, activation='relu', input_dim=latent_dim))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [41]:
def build_discriminator():
    model = tf.keras.Sequential()
    model.add(layers.Dense(512, activation='relu', input_dim=1))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [42]:
latent_dim = 100
generator = build_generator(latent_dim)
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy')
discriminator.trainable = False

In [43]:
discriminator.compile(optimizer='adam', loss='binary_crossentropy')
discriminator.trainable = False

In [44]:
gan_input = layers.Input(shape=(latent_dim,))
generated_note = generator(gan_input)
gan_output = discriminator(generated_note)
gan = tf.keras.Model(gan_input, gan_output)

gan.compile(optimizer='adam', loss='binary_crossentropy')

In [45]:
def train_gan(gan, generator, discriminator, data, epochs, batch_size, latent_dim):
    for epoch in range(epochs):
        idx = np.random.randint(0, data.shape[0], batch_size)
        real_notes = data[idx]
        
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        generated_notes = generator.predict(noise)
        
        combined_data = np.concatenate([real_notes, generated_notes])
        labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
        
        d_loss = discriminator.train_on_batch(combined_data, labels)
        
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        misleading_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, misleading_labels)
        
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}")

train_gan(gan, generator, discriminator, normalized_notes, epochs=1000, batch_size=64, latent_dim=latent_dim)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
Epoch 0, Discriminator Loss: 0.6909331679344177, Generator Loss: [array(0.69093317, dtype=float32), array(0.69093317, dtype=float32)]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step  
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16

In [46]:
def generate_music(generator, latent_dim):
    noise = np.random.normal(0, 1, (1, latent_dim))
    generated_notes = generator.predict(noise)
    return generated_notes

In [47]:
def denormalize_notes(notes, min_note, max_note):
    return [int(note * (max_note - min_note) + min_note) for note in notes]

In [48]:
generated_notes = generate_music(generator, latent_dim)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


In [49]:
denormalized_notes = denormalize_notes(generated_notes, min_note, max_note)

C:\Users\Student_B21\AppData\Local\Temp\ipykernel_4284\3250715730.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return [int(note * (max_note - min_note) + min_note) for note in notes]


In [52]:
def notes_to_midi(notes, output_file):
    midi = MidiFile()
    track = MidiTrack()
    midi.tracks.append(track)
    
    for note in notes:
        track.append(Message('note_on', note=note, velocity=64, time=480))
        track.append(Message('note_off', note=note, velocity=64, time=960))
    
    midi.save(output_file)

output_file = 'generated_music.mid'
notes_to_midi(denormalized_notes, output_file)